In [1]:
from multiprocessing import Pool

import ASWmultiprocess as PC

if __name__ == "__main__":
    
    # "Кнопки"
    t_air_list = [-29,-15, 0, 8, 15] # - температуры, при которых необходимо расчитать
    s = len(t_air_list)# - длина списка температур
    nagr_list = [1]# -нагрузки, при которых необходимо расчитать
    sn = len(nagr_list)
    
    # Флаги
    ASWbul = 0 #- 0- нет аккум 1- загрузка аккумулятора, 2- разгрузка аккумулятора
    ASWatm = False
    PKM_zaryad = False
    Сalculate_minimum = True
    Teplo = int(True)
    # Вкл выкл заряд по ПКМ
    PKM_zaryad = False

    
    #ASW konstruct parametr
    Diametr = 6
    Visota=10
    kolichestvo=2
    lambda_min_vata = 0.045
    delta_min_vata = 0.1
    vremya=4
    vremyaojidaniya = 12

    # Основные эффективности оборудования
    KPD_PN = 0.8074
    KPD_KN = 0.75
    KPD_to = 0.99
    KPD_SP = 0.99

    # Параметры, отвечающие за процесс расчета
    Calcmethod = "hybr"
    Calctolerance = 10**-2
    Maxiterations_KU_TU = 20
    Maxiterations_cotel = 4
    Maxiterations_turbine = 30

    # Отбор пара высокго давления или доля или кг/с
    steamVD_fraction_to_turbine = 1

    #####################
    
    chunks = [
        (
            Diametr,  # Diametr - диаметр аккумулятора
            kolichestvo,  # kolichestvo -  количество баков аккумуляторов
            Visota,  # Visota - высота аккумулятора
            lambda_min_vata,  # - lambda_min_vata теплопроводность изолятора бака аккумулятора
            delta_min_vata,  # - delta_min_vata толщина изолятора бака аккумулятора
            ASWbul,  # ASWbul - 1- загрузка аккумулятора, 2- разгрузка аккумулятора
            nagr_list[j],  # nagr - доля нагрузки от номинала
            vremya,  # vremya - время зарядки/разрядки аккумулятора
            vremyaojidaniya,  # vremyaojidaniya - время ожидания аккумулятора
            t_air_list[i],
            PKM_zaryad,
            Сalculate_minimum,
            KPD_PN,
            KPD_KN,
            KPD_to,
            KPD_SP,
            Calcmethod,
            Calctolerance,
            Maxiterations_KU_TU,
            Maxiterations_cotel,
            Maxiterations_turbine,
            Сalculate_minimum,
            Teplo,
            steamVD_fraction_to_turbine,
            PKM_zaryad,
            ASWatm,
        )
        for i in range(s)
        for j in range(sn)
    ]
    
    pool = Pool(processes=sn * s)
    all_results = pool.map(PC.ParallelCompute, chunks)

In [2]:
import pandas as pd
import openpyxl
import os
print(all_results)

df = pd.DataFrame(all_results)
df = df.T

if ASWbul==1:
    book = openpyxl.load_workbook('resdataASW.xlsx')
    writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df = pd.DataFrame(all_results)
    df.to_excel(writer, "Зарядка", startrow=0, startcol=0)
    writer.save()
if ASWbul==2:
    book = openpyxl.load_workbook('resdataASW.xlsx')
    writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df = pd.DataFrame(all_results)
    df.to_excel(writer, "Разрядка", startrow=0, startcol=0)
    writer.save()
if ASWbul==0:
    book = openpyxl.load_workbook('resdataASW.xlsx')
    writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df = pd.DataFrame(all_results)
    df.to_excel(writer, "нетаккум", startrow=0, startcol=0)
    writer.save()

[{'T_air': -29, 'n_GTU': 1, 'GTU': 142.3361, 'GTU_KPD': 32.7617, 'Turbine': 43.0733, 'KN': 0.0406, 'DK': 6.2519, 'PEN': 2.6282, 'Turbine_Qt': 157.0125, 'ASW_Qt': nan}, {'T_air': -15, 'n_GTU': 1, 'GTU': 157.1386, 'GTU_KPD': 33.8554, 'Turbine': 50.8335, 'KN': 0.0405, 'DK': 6.2833, 'PEN': 1.2982, 'Turbine_Qt': 168.1316, 'ASW_Qt': nan}, {'T_air': 0, 'n_GTU': 1, 'GTU': 152.7091, 'GTU_KPD': 33.5938, 'Turbine': 55.9455, 'KN': 0.0404, 'DK': 6.2739, 'PEN': 1.0771, 'Turbine_Qt': 167.5741, 'ASW_Qt': nan}, {'T_air': 8, 'n_GTU': 1, 'GTU': 145.9107, 'GTU_KPD': 33.2815, 'Turbine': 55.4008, 'KN': 0.0404, 'DK': 6.2595, 'PEN': 1.266, 'Turbine_Qt': 166.7372, 'ASW_Qt': nan}, {'T_air': 15, 'n_GTU': 1, 'GTU': 140.2577, 'GTU_KPD': 32.9477, 'Turbine': 55.0532, 'KN': 0.0404, 'DK': 6.2475, 'PEN': 1.4753, 'Turbine_Qt': 164.3898, 'ASW_Qt': nan}]


In [5]:
import mat_properties as prop
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
import numpy as np
from numpy import linalg as LA

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
Tpr_sv = 120
Tobr_sv=45
T_ASW=95
P_ASW=0.1
P_pr_sv = 1

G_ASW= 10

hpr_sv = water.p_t(P_pr_sv, Tpr_sv)['h']
print(hpr_sv,'hpr_sv')
hobr_sv = water.p_t(P_pr_sv, Tobr_sv)['h']
print(hobr_sv,'hobr_sv')
h_ASW = water.p_t(P_ASW, T_ASW)['h']
print(h_ASW,'h_ASW')

a = [[1, 1],
     [1*hpr_sv, 1*hobr_sv]]

b = [G_ASW, G_ASW*h_ASW]

x = LA.solve(a, b)
x


504.3750257054493 hpr_sv
189.30032512834947 hobr_sv
398.1007015109759 h_ASW


array([6.62701182, 3.37298818])